In [10]:
%%capture
# !pip install -U --force-reinstall sierra-ils-utils

# install from this branch for testing
# !pip install -U --force-reinstall git+https://github.com/chimpy-me/sierra-ils-utils.git@model/QueryEntry
# !pip install -U --force-reinstall git+https://github.com/chimpy-me/sierra-ils-utils.git

In [11]:
# !pip freeze | grep sierra-ils-utils

In [12]:
# import the modules
from sierra_ils_utils import SierraAPI, QueryBuilder
# from google.colab import userdata
import json
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

In [13]:
# from google.colab import userdata

with open('./.config.json') as f:
  config = json.load(f)

sierra_api = SierraAPI(
    # sierra_api_base_url=userdata.get('sierra_api_base_url'),
    sierra_api_base_url=config.get('sierra_api_base_url'),
    # sierra_api_key=userdata.get('sierra_api_key'),
    sierra_api_key=config.get('sierra_api_key'),
    # sierra_api_secret=userdata.get('sierra_api_secret'),
    sierra_api_secret=config.get('sierra_api_secret'),
    log_level=logging.DEBUG
)


# # setup the api client from sierra-ils-utils
# sierra_api = SierraAPI(
#     sierra_api_base_url=userdata.get('sierra-test_api_base_url'),
#     sierra_api_key=userdata.get('sierra-test_api_key'),
#     sierra_api_secret=userdata.get('sierra-test_api_secret'),
#     log_level='DEBUG'
# )

DEBUG:sierra_ils_utils.sierra_ils_utils:INIT {'base_url': 'https://classic.cincinnatilibrary.org/iii/sierra-api/v6/', 'api_key': '1ebmZG6N********************', 'request_count': 0, 'expires_at': 0, 'session_headers': {'accept': 'application/json', 'Authorization': ''}, 'endpoints': {'GET': {'bibs/': {'responses': {200: <class 'sierra_ils_utils.sierra_api_v6_endpoints.BibResultSet'>, 400: <class 'sierra_ils_utils.sierra_api_v6_endpoints.ErrorCode'>, 404: <class 'sierra_ils_utils.sierra_api_v6_endpoints.ErrorCode'>}, 'response_model': <class 'sierra_ils_utils.sierra_api_v6_endpoints.BibResultSet'>}, 'bibs/{id}': {'responses': {200: <class 'sierra_ils_utils.sierra_api_v6_endpoints.Bib'>, 400: <class 'sierra_ils_utils.sierra_api_v6_endpoints.ErrorCode'>, 404: <class 'sierra_ils_utils.sierra_api_v6_endpoints.ErrorCode'>}, 'response_model': <class 'sierra_ils_utils.sierra_api_v6_endpoints.Bib'>}, 'info/token': {'responses': {200: <class 'sierra_ils_utils.sierra_api_v6_endpoints.TokenInfo'>},

In [14]:
# we have a list of 100 item barcodes in a .csv file ...
# ... we're going to load them into a list

import csv

with open('./current_collection-a9be2bf.csv') as f:
    reader = csv.reader(f)             # use the csv reader to read the file
    columns = next(reader)             # first "row" is the column names
    rows = [row[0] for row in reader]  # produce a list of barcodes

print(
    columns,   # column names
    rows[:3],  # first 3 barcodes
    len(rows), # number of barcodes
    sep='\n'
)

['barcode']
['0923112556017', 'A000074528052', '8264518556018']
100


In [27]:
# sierra-ils-utils also has a (simple) query builder

# https://documentation.iii.com/sierrahelp/#sgil/sgil_lists_json.html
# > You can view and define Create Lists queries using JavaScript Object
# > Notation (JSON)

query = QueryBuilder()  # instatiate the class

# the query builder allows for "method chaining":
# ... below, we start a query with a target type of "item", and varfield tag "b"
# ... followed by the barcodes, "b" in the list of operands
# e.g. item records having a matching barcodes to ones in our list
query = query.start_query(record_type='item', field_tag='b') \
    .add_expression(op='in', operands=rows) \
    .end_query()  # end the query

note = """
this generates the following similar JSON by using the method, `query.json()`

{
  "target": {
    "record": {
      "type": "item"
    },
    "field": {
      "tag": "b"
    }
  },
  "expr": [
    {
      "op": "in",
      "operands": [
        "0923112556017",
        "A000074528052",
        "8264518556018"
      ]
    }
  ]
}
"""

In [28]:
# send the query!
response = sierra_api.post(
    'items/query',
    params={
        'limit':2000,
        'offset':0,
    },
    json_body=query.json()  # send the query as JSON
)

DEBUG:sierra_ils_utils.sierra_ils_utils:"template": items/query
DEBUG:sierra_ils_utils.sierra_ils_utils:POST {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/query", "params": "{'limit': 2000, 'offset': 0}", "json_body": "{'target': {'record': {'type': 'item'}, 'field': {'tag': 'b'}}, 'expr': [{'op': 'in', 'operands': ['0923112556017', 'A000074528052', '8264518556018', 'A000068621988', 'A000058040528', '1020643846014', '1062395-2001', '0995811352013', 'A000054341995', '8100145849014', 'A000023623580', 'A000075775231', 'A000074880321', '0937904152025', 'A000072564059', 'A000025464017', '0900611452138', 'A000064695739', 'A000011346970', '1023446846025', '0984097149012', 'A000011996113', 'A000065085880', 'A000063024121', 'A000071364972', 'A000051658912', 'A000012718318', '0985872058013', 'A000071899654', 'A000055732127', '1024650084022', 'A000014373419', '1004357046011', 'A000065073225', 'A000033164476', 'A000050379874', 'A000075409716', '1001787786018', '092214

In [18]:
print(
    len(response.data.entries),  # number of entries
    response.data.entries[:3],    # print the first 3 entries
    sep='\n'
)

100
[QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/1008510'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/1051453'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/1089999')]


In [19]:
# now that we have the items record numbers, we can get item data

result = sierra_api.get(
    'items/',
    params={
        'id': ','.join(response.data.entry_ids),
        'fields': 'default'
    }
)

DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/", "params": "{'id': '1008510,1051453,1089999,1128285,1149989,1310822,1601060,1793192,1968380,2006419,2020016,2020707,2030791,2056420,2093246,2268916,2315163,2320049,2374433,2599992,2638456,2665460,2860311,2991150,3113983,3144330,3148531,3191696,3247921,3727599,3791673,3831990,3956454,3975242,4060167,4196464,4254422,4387071,4782511,4799691,4897638,4911080,5174099,5280541,5293996,5465619,5552068,5579548,5626824,5729432,5776816,5949912,7761942,7871518,8101516,8208741,8423080,8603237,8605599,8677901,8736267,8980877,9059912,9222526,9250700,9451597,9539229,9677344,9678922,9943807,9960593,10135506,10238009,10265809,10401599,10420145,10421589,10456329,10616213,10669667,10779898,10938336,11035487,11140377,11291644,11346531,11461660,11496797,11503993,11506262,11540323,11622305,11742854,11824107,11860845,11896081,11955479,12104387,12108965,12161411', 'fields': 'default'}"}
DEBU

In [20]:
# result.data.entries[0]

# if items have a status->duedate, or volumes, don't place a hold on it.
items_for_holds = []
items_not_for_holds = []

for item in result.data.entries:
    if len(item.volumes) == 0 and item.status.duedate is None:
        # print(f'{item.id} PUT HOLD\n')
        items_for_holds.append(item.id)
    else:
        # print(f'{item.id} DON\'T PUT HOLD\n')
        items_not_for_holds.append(item.id)

In [21]:
len(items_for_holds), len(items_not_for_holds)

(41, 9)

In [22]:
item.status.duedate

In [23]:
# the create lists json query ...
json_body = """\
{
  "target": {
    "record": {"type": "bib"},
    "field": {"tag": "t"}
  },
  "expr": {
    "op": "has",
    "operands": ["spider-man"]
  }
}
"""

response = sierra_api.post(
    'bibs/query',
    params={
        'offset': 0,
        'limit': 10,
    },
    json_body=json_body
)

print(response)

DEBUG:sierra_ils_utils.sierra_ils_utils:"template": bibs/query
DEBUG:sierra_ils_utils.sierra_ils_utils:POST {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/query", "params": "{'offset': 0, 'limit': 10}", "json_body": "{'target': {'record': {'type': 'bib'}, 'field': {'tag': 't'}}, 'expr': {'op': 'has', 'operands': ['spider-man']}}"}
DEBUG:urllib3.connectionpool:Resetting dropped connection: classic.cincinnatilibrary.org
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "POST /iii/sierra-api/v6/bibs/query?offset=0&limit=10 HTTP/1.1" 200 834
DEBUG:sierra_ils_utils.sierra_ils_utils:"response.status_code": 200, "prepared_request.url": https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/query?offset=0&limit=10, "prepared_request.params": {"offset": 0, "limit": 10}, "prepared_request.body": {"target": {"record": {"type": "bib"}, "field": {"tag": "t"}}, "expr": {"op": "has", "operands": ["spider-man"]}}, "request_count": 4
DEBUG:sierra_i

"status_code"          : "200"
"response_model_name"  : "QueryResultSet"
"response_model_data"  : "{
    "total": 10,
    "start": 0,
    "entries": [
        {
            "link": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1061673"
        },
        {
            "link": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1069164"
        },
        {
            "link": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1382468"
        },
        {
            "link": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1527184"
        },
        {
            "link": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1643908"
        },
        {
            "link": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1789847"
        },
        {
            "link": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1806283"
        },
        {
            "link": "https://classic.cincinnatilibrary.

In [24]:
for entry in response.data.entry_ids:
  print(entry)

1061673
1069164
1382468
1527184
1643908
1789847
1806283
1859336
1871452
1886313


In [25]:
# print(response.data.entries)

In [26]:
if response.data.total and response.data.total > 0:
    for i, entry in enumerate(response.data.entries, start=1):
        print(i, entry.link)
        bib_response = sierra_api.get(
            'bibs/{id}',
            # params={
            #     'fields': 'default'
            # },
            path_params={
                'id': entry.link.split('/')[-1]  # just need the id or record num portion
            }
        )

        # print(bib_response.data.title)

DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1061673", "params": "{}"}
DEBUG:urllib3.connectionpool:Resetting dropped connection: classic.cincinnatilibrary.org


1 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1061673


DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/1061673 HTTP/1.1" 200 499
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 5
DEBUG:sierra_ils_utils.sierra_ils_utils:GET https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1061673 200 ✅
DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1069164", "params": "{}"}
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/1069164 HTTP/1.1" 200 460
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 6
DEBUG:sierra_ils_utils.sierra_ils_utils:GET https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1069164 200 ✅
DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1382468", "params": "{}"}
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/138246

2 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1069164
3 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1382468
4 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1527184


DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/1527184 HTTP/1.1" 200 452
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 8
DEBUG:sierra_ils_utils.sierra_ils_utils:GET https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1527184 200 ✅
DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1643908", "params": "{}"}
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/1643908 HTTP/1.1" 200 458
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 9
DEBUG:sierra_ils_utils.sierra_ils_utils:GET https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1643908 200 ✅
DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1789847", "params": "{}"}
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/178984

5 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1643908
6 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1789847
7 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1806283


DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/1806283 HTTP/1.1" 200 507
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 11
DEBUG:sierra_ils_utils.sierra_ils_utils:GET https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1806283 200 ✅
DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1859336", "params": "{}"}
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/1859336 HTTP/1.1" 200 512
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 12
DEBUG:sierra_ils_utils.sierra_ils_utils:GET https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1859336 200 ✅
DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1871452", "params": "{}"}
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/1871

8 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1859336
9 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1871452
10 https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1886313


DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/bibs/1886313 HTTP/1.1" 200 444
DEBUG:sierra_ils_utils.sierra_ils_utils:request count: 14
DEBUG:sierra_ils_utils.sierra_ils_utils:GET https://classic.cincinnatilibrary.org/iii/sierra-api/v6/bibs/1886313 200 ✅
